In [1]:
#librarys

import re
import os
import pandas as pd
import glob
import pandas
import html


In [2]:
#Creating a new excel file 
new_file = pd.ExcelWriter('demo1.xlsx', engine='xlsxwriter')
new_file.close()
new_file_count=0

#with open("Failed_list.txt", mode = "w") as f:
#    f.write("This text is written in python")
    
f= open("Failed_list.txt","w+")
f.close()

unknown= open("unknown_error_list.txt","w+")
unknown.close()

#column header list
new_list = []

#Success count values    
failed_count= 0
unknown_error= 0
string1_found =0
string2_found =0
string3_found =0
string4_found =0

In [3]:
def printing_data_into_excel(file_path):
    
    global new_file_count
    global writer
   
    global new_list #Column header
    
    global failed_count
    global unknown_error
    global string1_found
    global string2_found
    global string3_found
    global string4_found
    
    #Regex to remove the HTML from string
    CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

    def cleanhtml(raw_html):
        cleantext = re.sub(CLEANR, '', raw_html)
        return cleantext

    #Opening and reading a individual file
    with open(file_path,'r') as f:
        my_file = f.read()
        my_file = html.unescape(my_file)
        my_file = my_file.strip()
        
   
    try:
        #Extracting Company Name
        company_name = my_file.split('COMPANY CONFORMED NAME:', maxsplit=1)[1]\
                       .split('CENTRAL INDEX KEY:',maxsplit=1)[0]

        company_name_text= cleanhtml(company_name)

        #Extracting Company Name
        cik = my_file.split('CENTRAL INDEX KEY:', maxsplit=1)[1]\
                       .split('STANDARD INDUSTRIAL CLASSIFICATION:',maxsplit=1)[0]


        #Extracting FILED AS OF DATE
        Filed_as_date = my_file.split('FILED AS OF DATE:', maxsplit=1)[1]\
                       .split('DATE AS OF CHANGE:',maxsplit=1)[0]


        #Extracting DATE AS OF CHANGE
        Date_as_change = my_file.split('DATE AS OF CHANGE:', maxsplit=1)[1]\
                       .split('EFFECTIVENESS DATE:',maxsplit=1)[0]

        #Extracting Fiscal year end
        fiscal_year = my_file.split('FISCAL YEAR END:', maxsplit=1)[1]\
                       .split('FILING VALUES:',maxsplit=1)[0]
    
    #------------------------------------------------------------------------------------------------------------------#
    #------------------------------------------------------------------------------------------------------------------#

    
        
        string1 = "DIRECTOR COMPENSATION"
        string2 = "DIRECTORS AND EXECUTIVE" 
        string3 = "TRUSTEE COMPENSATION"
        string4 = "BENEFICIAL OWNERS AND MANAGEMENT"


        if re.search(string1, my_file,re.IGNORECASE):
            #print("Found!")
            string1_found=string1_found+1
            table_name = re.split(string1, my_file, flags=re.IGNORECASE, maxsplit=1)[1]

        elif re.search(string2, my_file,re.IGNORECASE):
            #print("found in 2!")
            string2_found=string2_found+1
            table_name = re.split(string2, my_file, flags=re.IGNORECASE, maxsplit=1)[1]
         

        elif re.search(string3, my_file,re.IGNORECASE):
            #print("found in 3!")
            string3_found=string3_found+1
            table_name = re.split(string3, my_file, flags=re.IGNORECASE, maxsplit=1)[1]
            

        elif re.search(string4, my_file,re.IGNORECASE):
            #print("found in 4!")
            string4_found=string4_found+1
            table_name = re.split(string4, my_file, flags=re.IGNORECASE, maxsplit=1)[1]
            

        else:
            failed_count = failed_count+1
            #print(failed_count)
            file1 = open("Failed_list.txt", "a")  # append mode
            file1.write(file_path +"\n")
            file1.close()
            return

    
        #Extracting Company Name
        

        #tables_on_page = pandas.read_html(table_name)
        tables_on_page = pd.read_html(table_name, match="(?i)^\s*Name\s*$")

        table = tables_on_page[0]

        # create a Dataframe
        Mydataframe = pd.DataFrame(table)

        nan_value = float("NaN")
        Mydataframe.replace(0, nan_value, inplace=True)
        Mydataframe.replace("", nan_value, inplace=True)
        #Mydataframe.replace("^[^A-Za-z0-9]$", nan_value, inplace=True, regex=True)

        my = Mydataframe.apply(lambda x: pd.Series(x.dropna().values))

        my.dropna(axis=1, inplace=True)

        new_header = my.iloc[0] #grab the first row for the header
        my = my[1:] #take the data less the header row
        my.columns = new_header #set the header row as the df header

        my.drop(my.tail(1).index,inplace=True)
        my.drop(my.head(1).index,inplace=True)

        company_name_add = company_name_text.strip()
        cik_add = cik.strip()
        Filed_as_date_add = Filed_as_date.strip()
        Date_as_change_add = Date_as_change.strip()
        fiscal_year_add = fiscal_year.strip()


        my.insert(0,'Company Name',[company_name_add] * len(my.index))
        my.insert(1,'CIK',[cik_add] * len(my.index))
        my.insert(2,'FILED AS OF DATE',[Filed_as_date_add] * len(my.index))
        my.insert(3,'DATE AS OF CHANGE',[Date_as_change_add] * len(my.index))
        my.insert(4,'FISCAL YEAR END',[fiscal_year_add] * len(my.index))

        #display(my)
        
        #Column name collection-------------------------------------------------------------------------------
        list_of_values = list(my.columns.values)

        for item in list_of_values:
            if item not in new_list:
                new_list.append(item)
        #---------------------------------------------------------------------------------------------------
        
        

        header_value=True

        if(new_file_count == 0):
            startrow = 0
            new_file_count=1

        else:
            sheet_max = writer.sheets['Sheet1'].max_row
            startrow = sheet_max+1
            #header_value=None

        with pd.ExcelWriter("demo1.xlsx",
            mode="a",
            engine="openpyxl",
            if_sheet_exists="overlay",
        ) as writer:
            my.to_excel(writer, sheet_name="Sheet1", startrow=startrow, header=header_value, index=False) 

    except IndexError:
        print("Key words not found")
    
    except :
        unknown_error = unknown_error+1
        print("Unknown error occurred")
        file_unknown = open("unknown_error_list.txt", "a")  # append mode
        file_unknown.write(file_path +"\n")
        file_unknown.close()
        
        
        

In [4]:
#Main program to call other functions
all_files = glob.glob(r"C:/Users/aniru/628_Anaconda/test1/**/*.txt",recursive=True)


for i in all_files:
    #print(i)
    printing_data_into_excel(i)

In [5]:
print("Failed runs:",failed_count)
print("Unknow error:",unknown_error)

print("String1 success:",string1_found)
print("String2 success:",string2_found)
print("String3 success:",string3_found)
print("String4 success:",string4_found)



Failed runs: 0
Unknow error: 0
String1 success: 1
String2 success: 0
String3 success: 0
String4 success: 0


In [11]:
#Column header collection and storage.

new_header_file = pd.ExcelWriter('header_list.xlsx', engine='xlsxwriter')
new_header_file.close()

#header_list = sorted(new_list)
header_list = new_list
header_dataframe = pd.DataFrame(header_list)


with pd.ExcelWriter("header_list.xlsx",
            mode="a",
            engine="openpyxl",
            if_sheet_exists="overlay",
        ) as writer:
            header_dataframe.to_excel(writer, sheet_name="Sheet1", startrow=0, header=None, index=False) 

In [10]:
#Currently working on this project-------------------------------------------------------------


# importing the modules
import pandas as pd
import numpy as np
 
# creating the DataFrame
my_data = {'Sr.no': [1, 2, 3, 4, 5],
           'Name': ['Ram', 'Sham', 'Sonu',
                    'Tinu', 'Monu'],
           'Maths Score': [45, 67, 89, 74, 56]}
df = pd.DataFrame(data = my_data)
 
# printing the original DataFrame
print("My Original DataFrame")
display(df)
 
list_sorter = ["Name","Sr.no"]
cols = list(df.columns)
#cols.reverse()
cols.sort(key=list_sorter.index)
 
# printing the new DataFrame
print("My new DataFrame")
 
df[cols]

My Original DataFrame


,Sr.no,Name,Maths Score
0,1,Ram,45
1,2,Sham,67
2,3,Sonu,89
3,4,Tinu,74
4,5,Monu,56


ValueError: 'Maths Score' is not in list

In [6]:
print(pd.__version__)


1.5.0
